In [2]:
import random

class Deck:
    faces = [2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A']
    suits = ['♦', '♣', '♠', '♥']
    
    def __init__(self, num_deck):
        self.cards = []
        self.num_deck = num_deck
        
    def createDeck(self):
        for i in self.faces:
            for j in self.suits:
                for num in range(self.num_deck):
                    self.cards.append((i,j))
        
        random.shuffle(self.cards)
        
    def dealCard(self):
        return self.cards.pop()
     
class Hand:
    def __init__(self):
        self.cards = []
        self.bet = 0
        
    def addCard(self, card):
        self.cards.append(card)
        
    def getScore(self):
        score = 0
        for index,card in enumerate(self.cards):
            if card[0] in range(2,11):
                score += self.cards[index][0]
            elif card[0] in ['J', 'Q', 'K']:
                score += 10
            elif card[0] == 'A':
                score += 11
                if score > 21:
                    score -= 10
        return score
    
    def displayCards(self):
        print(self.cards)
    
    def displayOneCard(self):
        print(self.cards[0])
        
    def resetCards(self):
        self.cards = []

    def setBet(self, amount):
        self.bet = amount
    
    def canSplit(self, player):
        return (self.cards[0][0] == self.cards[1][0] and
                player.budget >= self.bet*2)

class Player:
    def __init__(self, hand):
        self.budget = 500
        self.hands = [hand]
        
    def getBalance(self):
        return self.budget
    
    def setBalance(self, amount):
        self.budget += amount
        
class Game:
    def __init__(self, numOfDeck):
        self.myDeck = Deck(numOfDeck)
        self.inPlay = True
        
        # create deck
        self.myDeck.createDeck()
        
    def printInstruction(self):
        print('Welcome to BlackJack!!')
        print('============================')
        print('How to play:')
        print('1) Have a higher hand than the dealer to win.')
        print('2) Going over 21 is called a \'bust\' and you lose automatically.')
        print('3) The dealer must hit on any hand lower than 17.')
        print('4) If you and the dealer tie, it\'s a \'push\' and neither you nor the dealer loses money.')
        print('5) Type \'hit\' to receive a card from the deck on your turn.')
        print('6) Type \'stay\' to end your turn.')
        print('7) Type \'double\' to double down (once doubling down, your turn ends after receiving card).')
        print('8) Type \'split\' to split the cards in your hand (only allowed on pairs).')
        print('9) Each player begins with $500')
        print('10) The player with the highest amount during the session will be displayed when you leave the table.')
        print('11) Press \'q\' to quit at any time.')
    
    def hit(self, player, handIndex):
        card = self.myDeck.dealCard()
        player.hands[handIndex].addCard(card)
        player.hands[handIndex].displayCards()
        print(player.hands[handIndex].getScore())
        
        if player.hands[handIndex].getScore() > 21:
            print('You busted, HAHAHA!!!')
            player.setBalance(-player.hands[handIndex].bet)
            self.inPlay = False
        
    def stand(self, dealer):
        dealer_score = dealer.hands[0].getScore()
        
        while dealer_score < 17:
            card = self.myDeck.dealCard()
            dealer.hands[0].addCard(card)
        
            dealer_score = dealer.hands[0].getScore()
        
    def deal(self, player, dealer):
        for i in range(2):
            card = self.myDeck.dealCard()
            player.hands[0].addCard(card)
        
            card = self.myDeck.dealCard()
            dealer.hands[0].addCard(card)
        
        player.hands[0].displayCards()
        print(player.hands[0].getScore())
        
    def checkGame(self, player, dealer, handIndex):        
        print('Player cards: ', end='')
        player.hands[handIndex].displayCards()
        player_score = player.hands[handIndex].getScore()
        
        print('Dealer cards: ', end='')
        dealer.hands[0].displayCards()
        dealer_score = dealer.hands[0].getScore()
        
        print('Player score: \thand {}: {}'.format(handIndex + 1, player_score))           
        print('Dealer score: {}'.format(dealer_score))
        
        if (player_score == 21 or 
            (player_score > dealer_score and player_score <= 21 and dealer_score <= 21) or
            (player_score <= 21 and dealer_score > 21)):
            print('You win!!')
            dealer.setBalance(-player.hands[handIndex].bet)
            player.setBalance(player.hands[handIndex].bet)
        elif player_score < dealer_score and dealer_score <= 21:
            print('You busted, HAHAHA!!!')
            player.setBalance(-player.hands[handIndex].bet)
            dealer.setBalance(player.hands[handIndex].bet)
        elif ((player_score == 21 and dealer_score == 21) or 
            (player_score == dealer_score)):
            print('It\'s a push.')
    
        self.inPlay = False
    
    def checkBlackJack(self, player, dealer):
        player_score = player.hands[0].getScore()
        dealer_score = dealer.hands[0].getScore()
        
        if player_score == 21 or dealer_score == 21:
            if player_score == 21 and dealer_score == 21:
                print('It\'s a push.')
            elif player_score == 21:
                print('You win!!')
                dealer.setBalance(-player.hands[0].bet)
                player.setBalance(player.hands[0].bet)
            elif dealer_score == 21:
                print('You busted, HAHAHA!!!')
                player.setBalance(-player.hands[0].bet)
                dealer.setBalance(player.hands[0].bet)
                
            self.inPlay = False
        else:
            self.inPlay = True
    
    def split(self, player):
        new_hand = Hand()
        card = player.hands[0].cards.pop()
        new_hand.addCard(card)
        new_hand.setBet(player.hands[0].bet)
        
        return new_hand
        
    def double(self, player, handIndex):
        player.hands[handIndex].setBet(player.hands[handIndex].bet*2)
        self.hit(player, handIndex)

    def resetInPlay(self):
        self.inPlay = True
    
    def endPlay(self):
        self.inPlay = False

In [4]:
from IPython.display import clear_output

game = Game(3)    
player = Player(Hand())
dealer = Player(Hand())

splitReq = False
handsStatus = []

while True:
    game.printInstruction()
    
    print('')
    print('Your current balance is {}'.format(player.getBalance()))
    amount = int(input('How much would you like to bet? '))
    
    while amount > player.getBalance():
        amount = int(input('The bet amount is more than what the player can afford. Please enter new amount.'))

    player.hands[0].setBet(amount)
    
    game.deal(player, dealer)
    game.checkBlackJack(player, dealer)

    if game.inPlay:
        print('Dealer card: ', end='')
        dealer.hands[0].displayOneCard()
        while True:
            print('')
            ans = input('What would you like to do (type \'help\' for instructions)? ')
            if ans.lower() == 'split':
                if player.hands[0].canSplit(player):
                    splitReq = True
                    new_hand = game.split(player)
                    player.hands.append(new_hand)
                    
                    for i in range(len(player.hands)):
                        player.hands[i].displayCards()
                else:
                    print('You attempted to split a hand that was not eligible for a split.')
            elif ans.lower() == 'q':
                game.endPlay()
                break
            else:
                if splitReq and len(player.hands) > 0:
                    for i in range(len(player.hands)):
                        while True:
                            if i > 0:
                                print('')
                                ans = input('What would you like to do (type \'help\' for instructions)? ')
                            
                            player.hands[i].displayCards()
                            
                            if ans.lower() == 'help':
                                game.printInstruction()
                            elif ans.lower() == 'split':
                                print('Only allow split one time.')
                            elif ans.lower() == 'q':
                                game.endPlay()
                                break
                            elif ans.lower() == 'hit':
                                game.hit(player, i)
                                handsStatus.append(game.inPlay)
                                if not game.inPlay and i == 0:
                                    game.resetInPlay()
                                    break
                            elif ans.lower() == 'stay':
                                break
                            elif ans.lower() == 'double':
                                game.double(player, i)
                                break
                            
                            if i == 0:
                                print('')
                                ans = input('What would you like to do (type \'help\' for instructions)? ')

                else:
                    while True and ans.lower() != 'stay':
                        if ans.lower() == 'help':
                            game.printInstruction()
                        elif ans.lower() == 'split':
                            print('You attempted to split a hand that was not eligible for a split.')
                        elif ans.lower() == 'q':
                            game.endPlay()
                            break
                        elif ans.lower() == 'hit':
                            game.hit(player, 0)
                            if not game.inPlay:
                                break
                        elif ans.lower() == 'stay':
                            break
                        elif ans.lower() == 'double':
                            game.double(player, 0)
                            break

                        print('')
                        ans = input('What would you like to do (type \'help\' for instructions)? ')
                
                break
        
        if len(handsStatus) > 0:
            for index in range(len(handsStatus)):
                if handsStatus[index]:
                    game.stand(dealer)
                    game.checkGame(player, dealer, index)
        else:
            if game.inPlay:
                game.stand(dealer)
                game.checkGame(player, dealer, 0)

    replay = input('Would you like to play again (y/n)? ')
    if replay.lower() == 'n':
        break
    elif replay.lower() == 'y' and (player.getBalance() <= 0 or game.myDeck.cards == [] or dealer.getBalance() <= 0):
        print('Sorry, try again next time!')
        break

    if len(player.hands) > 1:
        player.hands.pop()

    player.hands[0].resetCards()  
    dealer.hands[0].resetCards()
    game.resetInPlay()
    splitReq = False
    handsStatus = []
    
    clear_output()

if player.getBalance() > dealer.getBalance():
    print('You have the highest amount: {}'.format(player.getBalance()))
else:
    print('Dealer has the highest amount: {}'.format(dealer.getBalance()))

Welcome to BlackJack!!
How to play:
1) Have a higher hand than the dealer to win.
2) Going over 21 is called a 'bust' and you lose automatically.
3) The dealer must hit on any hand lower than 17.
4) If you and the dealer tie, it's a 'push' and neither you nor the dealer loses money.
5) Type 'hit' to receive a card from the deck on your turn.
6) Type 'stay' to end your turn.
7) Type 'double' to double down (once doubling down, your turn ends after receiving card).
8) Type 'split' to split the cards in your hand (only allowed on pairs).
9) Each player begins with $500
10) The player with the highest amount during the session will be displayed when you leave the table.
11) Press 'q' to quit at any time.

Your current balance is 600
How much would you like to bet? 1
[(2, '♠'), (3, '♦')]
5
Dealer card: (9, '♠')

What would you like to do (type 'help' for instructions)? hit
[(2, '♠'), (3, '♦'), ('K', '♣')]
15

What would you like to do (type 'help' for instructions)? hit
[(2, '♠'), (3, '♦'),